In [132]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

# Set up Chrome options
options = webdriver.ChromeOptions()

# Create a new Chrome session
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

try:
    # Step 1: Log in to the university portal
    university_login_url = 'https://uaccess.univie.ac.at'
    driver.get(university_login_url)
    time.sleep(1)  # Wait for the page to load

    # Enter university login credentials
    driver.find_element(By.ID, 'userid').send_keys("")                                       # <- enter user                                                           # Replace with actual element ID
    driver.find_element(By.ID, 'password').send_keys("")                                     # <- enter password                                              # Replace with actual element ID
    driver.find_element(By.NAME, '_eventId_proceed').click()                                                                        # Replace with actual element ID
    time.sleep(1)  # Wait for the page to load

    # Step 2: Log in to CapitalIQ
    capitaliq_login_url = 'https://www-capitaliq-spglobal-com.uaccess.univie.ac.at/web/client?auth=inherit#dashboard/sfi'
    driver.get(capitaliq_login_url)
    time.sleep(7)  # Wait for the page to load

    # Enter email address and navigate to the next step
    driver.find_element(By.ID, 'input28').send_keys("")                                      # <- enter user                                        # Replace with actual element ID
    driver.find_element(By.XPATH, "//input[@type='submit' and @value='Next']").click()                                              # Replace with actual element ID
    time.sleep(2)  # Wait for the page to load

    # Enter password and navigate to the next step
    driver.find_element(By.ID, 'input59').send_keys("")                                      # <- enter password                                                                   # Replace with actual element ID
    driver.find_element(By.XPATH, "//input[@type='submit' and @value='Sign In']").click()                                           # Replace with actual element ID
    time.sleep(2)  # Wait for the page to load
        
    # Step 3: Navigate to the desired page
    final_url = 'https://www-capitaliq-spglobal-com.uaccess.univie.ac.at/web/client?auth=inherit#dashboard/sfi'
    driver.get(final_url)
    time.sleep(2)  # Wait for the page to load

finally:
    # Close the browser when done
    1 #driver.quit()


In [186]:
# Handle cookie consent if it appears

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def accept_cookies(waittime = 5):
    try:
        accept_cookies = WebDriverWait(driver, waittime).until(
             EC.presence_of_element_located((By.ID, "onetrust-accept-btn-handler")))
        time.sleep(2)
        accept_cookies.click()
    except:
        print("Cookie consent button not found or already accepted")

accept_cookies()

In [134]:
# Get company sample

import pandas as pd

# Read the Excel file into a DataFrame
df = pd.read_excel("companies.xlsx", engine='openpyxl')

# Display the first few rows of the DataFrame
df.head()

,Entity Name,Entity ID,Ticker,Country / Region Name,Market Capitalization\n($M),Market Capitalization\n($M).1,Market Capitalization\n($M).2
0,Lion Rock Resources Inc. (TSXV:ROAR),4351623,ROAR,Canada,4.919976,19.99,3.694655
1,Thor Energy Plc (AIM:THR),4354834,THR,United Kingdom,4.916056,19.98,6.842020
2,QX Resources Limited (ASX:QXR),4348602,QXR,Australia,4.890542,19.94,11.341354
3,Battery X Metals Inc. (CNSX:BATX),19420976,BATX,Canada,4.879013,19.91,2.551188
4,Silver Wolf Exploration Ltd. (TSXV:SWLF),4360301,SWLF,Canada,4.875764,19.90,2.031725


In [136]:
# search and scrape

#companies = df.head(10)["Entity ID"].astype("str")
companies = df["Entity ID"].astype("str")

companies_with_news = []
comps_and_links = {}

for company in companies:
    url = "https://www-capitaliq-spglobal-com.uaccess.univie.ac.at/web/client#company/briefingbooknews?ID=" + company


    # Wait for the "News" link to be present in the expanded menu
    try:
        driver.get(url)
        time.sleep(4)                                                   #[!!!!!!] increase the time if it complaints [!!!!!!]

        # Locate the section containing the links
        section = driver.find_element(By.CSS_SELECTOR, "tbody.ui-widget-content.ui-iggrid-tablebody")
    
        # Find all <a> tags within the section
        links = section.find_elements(By.TAG_NAME, 'a')
    
        # Create a list to store the href attributes of the links
        links_list = [link.get_attribute('href') for link in links if link.get_attribute('href') is not None]
        
        # Print the list of links
        print(company, "- News link clicked")
        print(links_list)

        # Append comps_and_links
        if links_list:
            companies_with_news.append(company)
            comps_and_links[company] = links_list


    
    except:
        print(company, "- News link not found")

    


    

4351623 - News link not found
4354834 - News link clicked
['https://www-capitaliq-spglobal-com.uaccess.univie.ac.at/apisv3/spg-webplatform-core/news/article?Id=80761965&KeyProductLinkType=2', 'https://www-capitaliq-spglobal-com.uaccess.univie.ac.at/apisv3/spg-webplatform-core/news/article?Id=80755463&KeyProductLinkType=2', 'https://www-capitaliq-spglobal-com.uaccess.univie.ac.at/apisv3/spg-webplatform-core/news/article?Id=80750780&KeyProductLinkType=2', 'https://www-capitaliq-spglobal-com.uaccess.univie.ac.at/apisv3/spg-webplatform-core/news/article?Id=77622224&KeyProductLinkType=2', 'https://www-capitaliq-spglobal-com.uaccess.univie.ac.at/apisv3/spg-webplatform-core/news/article?Id=77548053&KeyProductLinkType=2', 'https://www-capitaliq-spglobal-com.uaccess.univie.ac.at/apisv3/spg-webplatform-core/news/article?Id=77518082&KeyProductLinkType=2', 'https://www-capitaliq-spglobal-com.uaccess.univie.ac.at/apisv3/spg-webplatform-core/news/article?Id=71575299&KeyProductLinkType=2']
4348602 - 

In [137]:
print(companies_with_news)
#print(comps_and_links)

['4354834', '4353862', '4353721', '10544220', '105895532', '4352871', '4864438', '4350189', '5578992', '106597483', '4625498', '4356762', '4201620', '4360321', '4351396', '29743046', '29200129', '4349571', '4349941', '25814263', '4352870', '8005606', '4354804', '4981700', '4354712', '4533656', '15899869', '4259601', '4353698', '4354464', '4350183', '4983358', '29166773', '27801629', '4444995', '4362365', '9381427', '28250936', '4544567', '4349433', '4348562', '4351833', '19950919', '4260464', '4349611', '4353371', '103006', '4353419', '105924909', '4367824', '4225187', '105979258', '4352914', '26637064', '4349354', '106153860', '4351531', '4353249', '29495491', '4353070', '101304688', '4348327', '4248016', '4351361', '4357196', '4973340', '4352738', '105759522', '4352435', '4121704', '4355238', '6350494', '4261144', '4348547', '4231824', '29721525', '4350616', '4163935', '8308166', '100518188', '4772176', '4349428', '100433', '4347817', '4352752', '4353242', '10825361', '4350180', '261

In [142]:
# save the companies and links to articles as a json file 
import json

file = open("comps_and_links.json", "w")
json.dump(comps_and_links, file)
file.close()

In [277]:
# check if it saved correctly 
file2 = open("comps_and_links.json", "r")
data = json.load(file2)
file2.close()

print(data)

{'4354834': ['https://www-capitaliq-spglobal-com.uaccess.univie.ac.at/apisv3/spg-webplatform-core/news/article?Id=80761965&KeyProductLinkType=2', 'https://www-capitaliq-spglobal-com.uaccess.univie.ac.at/apisv3/spg-webplatform-core/news/article?Id=80755463&KeyProductLinkType=2', 'https://www-capitaliq-spglobal-com.uaccess.univie.ac.at/apisv3/spg-webplatform-core/news/article?Id=80750780&KeyProductLinkType=2', 'https://www-capitaliq-spglobal-com.uaccess.univie.ac.at/apisv3/spg-webplatform-core/news/article?Id=77622224&KeyProductLinkType=2', 'https://www-capitaliq-spglobal-com.uaccess.univie.ac.at/apisv3/spg-webplatform-core/news/article?Id=77548053&KeyProductLinkType=2', 'https://www-capitaliq-spglobal-com.uaccess.univie.ac.at/apisv3/spg-webplatform-core/news/article?Id=77518082&KeyProductLinkType=2', 'https://www-capitaliq-spglobal-com.uaccess.univie.ac.at/apisv3/spg-webplatform-core/news/article?Id=71575299&KeyProductLinkType=2'], '4353862': ['https://www-capitaliq-spglobal-com.uaccess

In [165]:
# OK, I think, I forgot to include page iteration for campanies with more than 20 articles

comps_and_link_counts = {company: len(links) for company, links in comps_and_links.items()}
subset_20_links = {company: count for company, count in comps_and_link_counts.items() if count == 20}
for i in subset_20_links:
    print(i)

4259601
105979258
4248016
4163935
100433
4351090
7696016
4051306
4090546
100298


In [191]:
# This will iterate over the pages:
import math

for company in subset_20_links:

    url = "https://www-capitaliq-spglobal-com.uaccess.univie.ac.at/web/client#company/briefingbooknews?ID=" + company
    
    try:
        # Open the initial URL where the pagination is located
        driver.get(url)
        accept_cookies(4) # already includes a waiting period
        #time.sleep(6)  # Wait for the page to load
        
        # Locate the element containing the total number of records
        records_label = driver.find_element(By.CSS_SELECTOR, ".ui-iggrid-pagerrecordslabel.ui-iggrid-results")
        total_records_text = records_label.text
        print(total_records_text)
        total_records = int(total_records_text.split('of ')[1].split(' ')[0].replace(',', ''))
        
        # Calculate the total number of pages (20 records per page)
        records_per_page = 20
        total_pages = math.ceil(total_records / records_per_page)
        
        print(f"Total pages: {total_pages}")
        
        for page in range(1, total_pages + 1):
            print(f"Processing page {page}")
            
            # Scrape data from the current page part
            # data storing part
    
            # If not the last page, click the "Next" button to go to the next page
            if page < total_pages:
                next_button = driver.find_element(By.XPATH, "//div[contains(@id, 'section_1_control_') and contains(@id, '_grid_table_next_page')]")
                next_button.click()
                time.sleep(4)  # Wait for the next page to load

    finally:
        # Close the browser when done
        1 #driver.quit()


1 - 20 of 340 records
Total pages: 17
Processing page 1
Processing page 2
Processing page 3
Processing page 4
Processing page 5
Processing page 6
Processing page 7
Processing page 8
Processing page 9
Processing page 10
Processing page 11
Processing page 12
Processing page 13
Processing page 14
Processing page 15
Processing page 16
Processing page 17
Cookie consent button not found or already accepted
1 - 20 of 21 records
Total pages: 2
Processing page 1
Processing page 2
Cookie consent button not found or already accepted
1 - 20 of 20 records
Total pages: 1
Processing page 1
Cookie consent button not found or already accepted
1 - 20 of 52 records
Total pages: 3
Processing page 1
Processing page 2
Processing page 3
Cookie consent button not found or already accepted
1 - 20 of 244 records
Total pages: 13
Processing page 1
Processing page 2
Processing page 3
Processing page 4
Processing page 5
Processing page 6
Processing page 7
Processing page 8
Processing page 9
Processing page 10
Proce

In [280]:
# This will iterate over the pages:
import math

companies_with_news = []
comps_and_links = {}

for company in subset_20_links:

    url = "https://www-capitaliq-spglobal-com.uaccess.univie.ac.at/web/client#company/briefingbooknews?ID=" + company
    links_list = []
    
    try:
        # Open the initial URL where the pagination is located
        driver.get(url)
        accept_cookies(4) # already includes a waiting period
        #time.sleep(6)  # Wait for the page to load
        
        # Locate the element containing the total number of records
        records_label = driver.find_element(By.CSS_SELECTOR, ".ui-iggrid-pagerrecordslabel.ui-iggrid-results")
        total_records_text = records_label.text
        print(total_records_text)
        total_records = int(total_records_text.split('of ')[1].split(' ')[0].replace(',', ''))
        
        # Calculate the total number of pages (20 records per page)
        records_per_page = 20
        total_pages = math.ceil(total_records / records_per_page)
        
        print(f"Total pages: {total_pages}")
        
        for page in range(1, total_pages + 1):
            print(f"Processing page {page}")
            
            # Locate the section containing the links
            section = driver.find_element(By.CSS_SELECTOR, "tbody.ui-widget-content.ui-iggrid-tablebody")
        
            # Find all <a> tags within the section
            links = section.find_elements(By.TAG_NAME, 'a')
        
            # Create a list to store the href attributes of the links
            links_list_this_page = [link.get_attribute('href') for link in links if link.get_attribute('href') is not None]
            links_list = links_list + links_list_this_page
    
            # Append comps_and_links
            if links_list:
                companies_with_news.append(company)
                
    
            # If not the last page, click the "Next" button to go to the next page
            if page < total_pages:
                next_button = driver.find_element(By.XPATH, "//div[contains(@id, 'section_1_control_') and contains(@id, '_grid_table_next_page')]")
                next_button.click()
                time.sleep(4)  # Wait for the next page to load
       
    
        comps_and_links[company] = links_list
    
        # Print the list of links
        print(company, "- News link clicked - ", len(links_list))

    except:
        print(company, "- News link not found")


1 - 20 of 340 records
Total pages: 17
Processing page 1
Processing page 2
Processing page 3
Processing page 4
Processing page 5
Processing page 6
Processing page 7
Processing page 8
Processing page 9
Processing page 10
Processing page 11
Processing page 12
Processing page 13
Processing page 14
Processing page 15
Processing page 16
Processing page 17
4259601 - News link clicked -  340
Cookie consent button not found or already accepted
1 - 20 of 22 records
Total pages: 2
Processing page 1
Processing page 2
105979258 - News link clicked -  22
Cookie consent button not found or already accepted
1 - 20 of 20 records
Total pages: 1
Processing page 1
4248016 - News link clicked -  20
Cookie consent button not found or already accepted
1 - 20 of 52 records
Total pages: 3
Processing page 1
Processing page 2
Processing page 3
4163935 - News link clicked -  52
Cookie consent button not found or already accepted
1 - 20 of 244 records
Total pages: 13
Processing page 1
Processing page 2
Processing 

In [281]:
#Check the number of links
for key, links in comps_and_links.items():
    print(f"{key} - Number of links: {len(links)}")

4259601 - Number of links: 340
105979258 - Number of links: 22
4248016 - Number of links: 20
4163935 - Number of links: 52
100433 - Number of links: 244
4351090 - Number of links: 84
7696016 - Number of links: 28
4051306 - Number of links: 28
4090546 - Number of links: 41
100298 - Number of links: 101


In [198]:
# Check handling of empty news:
import math

companies = df.head(10)["Entity ID"].astype("str")

companies_with_news = []
comps_and_links = {}

for company in companies:

    url = "https://www-capitaliq-spglobal-com.uaccess.univie.ac.at/web/client#company/briefingbooknews?ID=" + company
    links_list = []
    
    try:
        # Open the initial URL where the pagination is located
        driver.get(url)
        accept_cookies(4) # already includes a waiting period
        #time.sleep(6)  # Wait for the page to load
        
        # Locate the element containing the total number of records
        records_label = driver.find_element(By.CSS_SELECTOR, ".ui-iggrid-pagerrecordslabel.ui-iggrid-results")
        total_records_text = records_label.text
        print(total_records_text)
        total_records = int(total_records_text.split('of ')[1].split(' ')[0].replace(',', ''))
        
        # Calculate the total number of pages (20 records per page)
        records_per_page = 20
        total_pages = math.ceil(total_records / records_per_page)
        
        print(f"Total pages: {total_pages}")
        
        for page in range(1, total_pages + 1):
            print(f"Processing page {page}")
            
            # Locate the section containing the links
            section = driver.find_element(By.CSS_SELECTOR, "tbody.ui-widget-content.ui-iggrid-tablebody")
        
            # Find all <a> tags within the section
            links = section.find_elements(By.TAG_NAME, 'a')
        
            # Create a list to store the href attributes of the links
            links_list_this_page = [link.get_attribute('href') for link in links if link.get_attribute('href') is not None]
            links_list = links_list + links_list_this_page
    
            # Append comps_and_links
            if links_list:
                companies_with_news.append(company)
                
    
            # If not the last page, click the "Next" button to go to the next page
            if page < total_pages:
                next_button = driver.find_element(By.XPATH, "//div[contains(@id, 'section_1_control_') and contains(@id, '_grid_table_next_page')]")
                next_button.click()
                time.sleep(4)  # Wait for the next page to load
       
    
        comps_and_links[company] = links_list
    
        # Print the list of links
        print(company, "- News link clicked - ", len(links_list))

    except:
        print(company, "- News link not found")


Cookie consent button not found or already accepted

4351623 - News link not found
Cookie consent button not found or already accepted
1 - 7 of 7 records
Total pages: 1
Processing page 1
4354834 - News link clicked -  7
Cookie consent button not found or already accepted
4348602 - News link not found
Cookie consent button not found or already accepted
19420976 - News link not found
Cookie consent button not found or already accepted
4360301 - News link not found
Cookie consent button not found or already accepted
4864489 - News link not found
Cookie consent button not found or already accepted
4361965 - News link not found
Cookie consent button not found or already accepted
1 - 5 of 5 records
Total pages: 1
Processing page 1
19689543 - News link clicked -  5
Cookie consent button not found or already accepted
26395223 - News link not found
Cookie consent button not found or already accepted
27709705 - News link not found


In [199]:
# Works perfectly
comps_and_links

{'4354834': ['https://www-capitaliq-spglobal-com.uaccess.univie.ac.at/apisv3/spg-webplatform-core/news/article?Id=80761965&KeyProductLinkType=2',
  'https://www-capitaliq-spglobal-com.uaccess.univie.ac.at/apisv3/spg-webplatform-core/news/article?Id=80755463&KeyProductLinkType=2',
  'https://www-capitaliq-spglobal-com.uaccess.univie.ac.at/apisv3/spg-webplatform-core/news/article?Id=80750780&KeyProductLinkType=2',
  'https://www-capitaliq-spglobal-com.uaccess.univie.ac.at/apisv3/spg-webplatform-core/news/article?Id=77622224&KeyProductLinkType=2',
  'https://www-capitaliq-spglobal-com.uaccess.univie.ac.at/apisv3/spg-webplatform-core/news/article?Id=77548053&KeyProductLinkType=2',
  'https://www-capitaliq-spglobal-com.uaccess.univie.ac.at/apisv3/spg-webplatform-core/news/article?Id=77518082&KeyProductLinkType=2',
  'https://www-capitaliq-spglobal-com.uaccess.univie.ac.at/apisv3/spg-webplatform-core/news/article?Id=71575299&KeyProductLinkType=2'],
 '19689543': ['https://www-capitaliq-spglob

In [201]:
# The function for future use if we even need more companies:

# All libraries above (I'm NOT figuring out, which are irrelevant here)
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import math
import pandas as pd
import json




def get_news_links(companies): 
    
    companies = companies.astype("str")
    
    companies_with_news = []
    comps_and_links = {}
    
    for company in companies:
    
        url = "https://www-capitaliq-spglobal-com.uaccess.univie.ac.at/web/client#company/briefingbooknews?ID=" + company
        links_list = []
        
        try:
            # Open the initial URL where the pagination is located
            driver.get(url)
            accept_cookies(4) # already includes a waiting period
            #time.sleep(6)  # Wait for the page to load
            
            # Locate the element containing the total number of records
            records_label = driver.find_element(By.CSS_SELECTOR, ".ui-iggrid-pagerrecordslabel.ui-iggrid-results")
            total_records_text = records_label.text
            print(total_records_text)
            total_records = int(total_records_text.split('of ')[1].split(' ')[0].replace(',', ''))
            
            # Calculate the total number of pages (20 records per page)
            records_per_page = 20
            total_pages = math.ceil(total_records / records_per_page)
            
            print(f"Total pages: {total_pages}")
            
            for page in range(1, total_pages + 1):
                print(f"Processing page {page}")
                
                # Locate the section containing the links
                section = driver.find_element(By.CSS_SELECTOR, "tbody.ui-widget-content.ui-iggrid-tablebody")
            
                # Find all <a> tags within the section
                links = section.find_elements(By.TAG_NAME, 'a')
            
                # Create a list to store the href attributes of the links
                links_list_this_page = [link.get_attribute('href') for link in links if link.get_attribute('href') is not None]
                links_list = links_list + links_list_this_page
        
                # Append comps_and_links
                if links_list:
                    companies_with_news.append(company)
                    
        
                # If not the last page, click the "Next" button to go to the next page
                if page < total_pages:
                    next_button = driver.find_element(By.XPATH, "//div[contains(@id, 'section_1_control_') and contains(@id, '_grid_table_next_page')]")
                    next_button.click()
                    time.sleep(4)  # Wait for the next page to load
           
        
            comps_and_links[company] = links_list
        
            # Print the list of links
            print(company, "- News link clicked - ", len(links_list))
    
        except:
            print(company, "- News link not found")




    return companies_with_news, comps_and_links

In [215]:
# Check how it works:

comps, links_dict = get_news_links(df.head(10)["Entity ID"])

4351623 - News link not found
Cookie consent button not found or already accepted
1 - 7 of 7 records
Total pages: 1
Processing page 1
4354834 - News link clicked -  7
Cookie consent button not found or already accepted
4348602 - News link not found
Cookie consent button not found or already accepted
19420976 - News link not found
Cookie consent button not found or already accepted
4360301 - News link not found
Cookie consent button not found or already accepted
4864489 - News link not found
Cookie consent button not found or already accepted
4361965 - News link not found
Cookie consent button not found or already accepted
1 - 5 of 5 records
Total pages: 1
Processing page 1
19689543 - News link clicked -  5
Cookie consent button not found or already accepted
26395223 - News link not found
Cookie consent button not found or already accepted
27709705 - News link not found


In [205]:
# Check:
comps

['4354834', '19689543']

In [206]:
# Check:
links_dict

{'4354834': ['https://www-capitaliq-spglobal-com.uaccess.univie.ac.at/apisv3/spg-webplatform-core/news/article?Id=80761965&KeyProductLinkType=2',
  'https://www-capitaliq-spglobal-com.uaccess.univie.ac.at/apisv3/spg-webplatform-core/news/article?Id=80755463&KeyProductLinkType=2',
  'https://www-capitaliq-spglobal-com.uaccess.univie.ac.at/apisv3/spg-webplatform-core/news/article?Id=80750780&KeyProductLinkType=2',
  'https://www-capitaliq-spglobal-com.uaccess.univie.ac.at/apisv3/spg-webplatform-core/news/article?Id=77622224&KeyProductLinkType=2',
  'https://www-capitaliq-spglobal-com.uaccess.univie.ac.at/apisv3/spg-webplatform-core/news/article?Id=77548053&KeyProductLinkType=2',
  'https://www-capitaliq-spglobal-com.uaccess.univie.ac.at/apisv3/spg-webplatform-core/news/article?Id=77518082&KeyProductLinkType=2',
  'https://www-capitaliq-spglobal-com.uaccess.univie.ac.at/apisv3/spg-webplatform-core/news/article?Id=71575299&KeyProductLinkType=2'],
 '19689543': ['https://www-capitaliq-spglob

In [302]:
# NOW, we need to get the articles themselves:
# This time I will start with a function from the get-go
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import re

# Dictionary of company codes and URLs
companies = links_dict



# Function to find the content area using multiple selectors
def find_content_area(driver):
    selectors = [
        {'by': By.CSS_SELECTOR, 'value': 'div[data-testid="newsArticleDJ"]'},
        {'by': By.ID, 'value': 'lazy-article-div'}
    ]
    for selector in selectors:
        try:
            element = driver.find_element(selector['by'], selector['value'])
            if element:
                return element
        except:
            continue
    return None  # Return None if no content area is found
    
# Function to extract article text using multiple possible selectors
def extract_article_text(driver):
    selectors = [
        {'by': By.CSS_SELECTOR, 'value': 'div[data-testid="ArticleContent_0"]'},
        {'by': By.CSS_SELECTOR, 'value': 'div[id="newsArticleMI"]'}
    ]
    for selector in selectors:
        try:
            element = driver.find_element(selector['by'], selector['value'])
            if element:
                return element.text
        except:
            continue
    return ""  # Return empty string if no article content is found

# Regular expression pattern for dates
date_pattern = re.compile(r'\b(?:Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday), \b(?:January|February|March|April|May|June|July|August|September|October|November|December) \d{1,2}, \d{4} \d{1,2}:\d{2} (AM|PM) ET')

# Function to extract the first matching date
def extract_date(content_area):
    texts = content_area.find_elements(By.XPATH, ".//*[contains(@class, 'spg-text-secondary')]")
    for text in texts:
        match = date_pattern.search(text.text)
        if match:
            return match.group()
    return ""  # Return empty string if no date is found

# Iterate over companies and their URLs
def database_composition(companies):
    # Initialize a list to store data
    data = []
    
    total_links = sum(len(links) for links in companies.values())
    k = 0
    
    for company_code, urls in companies.items():
        for url in urls:

            k = k + 1
            print(str(k), "/", str(total_links))
            
            driver.get(url)
            time.sleep(4)  # Wait for the page to load completely
    
            # Attempt to find the content area
            content_area = find_content_area(driver)
            if not content_area:
                print("bad content area for:   ", url)
                continue  # Skip this URL if the content area is not found
    
            # Extract all titles and concatenate them within the content area
            titles = content_area.find_elements(By.CLASS_NAME, "ignore-for-localizejs")
            title_text = "; ".join(title.text for title in titles)
    
            
    
            # Extract the first publication date
            publication_date = extract_date(content_area)
            if publication_date == "":
                print("bad date for:   ", url)
                
            # Extract the article text
            article_text = extract_article_text(driver)
    
            # Append the extracted data to the list
            data.append([company_code, url, title_text, publication_date, article_text])
    
    # Create a DataFrame
    final_data = pd.DataFrame(data, columns=["company_code", "url", "title", "publication_date", "article"])
    
    # Save the DataFrame to CSV
    final_data.to_csv("company_articles.csv", index=False)
    
    # Close the WebDriver
    #driver.quit()
    
    print("Data extraction and saving complete.")

In [295]:
# Correct links for companies with >20 links:
for company in comps_and_links:
    data[company] = comps_and_links[company]

In [303]:
# Making of a databse

companies_for_db = data

database_composition(data)

1 / 1325
2 / 1325
3 / 1325
4 / 1325
5 / 1325
6 / 1325
7 / 1325
8 / 1325
9 / 1325
10 / 1325
11 / 1325
12 / 1325
13 / 1325
14 / 1325
15 / 1325
16 / 1325
17 / 1325
18 / 1325
19 / 1325
20 / 1325
21 / 1325
22 / 1325
23 / 1325
24 / 1325
25 / 1325
26 / 1325
27 / 1325
28 / 1325
29 / 1325
30 / 1325
31 / 1325
32 / 1325
33 / 1325
34 / 1325
35 / 1325
36 / 1325
37 / 1325
38 / 1325
39 / 1325
40 / 1325
41 / 1325
42 / 1325
43 / 1325
44 / 1325
45 / 1325
46 / 1325
47 / 1325
48 / 1325
49 / 1325
50 / 1325
51 / 1325
52 / 1325
53 / 1325
bad content area for:    https://www-capitaliq-spglobal-com.uaccess.univie.ac.at/apisv3/spg-webplatform-core/news/article?Id=76635472&KeyProductLinkType=2
54 / 1325
55 / 1325
56 / 1325
57 / 1325
58 / 1325
59 / 1325
60 / 1325
61 / 1325
62 / 1325
63 / 1325
64 / 1325
65 / 1325
66 / 1325
67 / 1325
68 / 1325
69 / 1325
70 / 1325
71 / 1325
72 / 1325
73 / 1325
74 / 1325
75 / 1325
76 / 1325
77 / 1325
78 / 1325
79 / 1325
80 / 1325
81 / 1325
82 / 1325
83 / 1325
84 / 1325
85 / 1325
86 /

In [351]:
# Load the data 
final_data = pd.read_csv("company_articles.csv")
print(final_data.iloc[630:650, 3])

630          Thursday, May 30, 2024 12:28 AM ET
631    Thursday, September 28, 2023 12:49 AM ET
632    Wednesday, September 27, 2023 1:06 AM ET
633     Tuesday, September 26, 2023 12:45 AM ET
634       Monday, September 25, 2023 1:09 AM ET
635      Friday, September 22, 2023 12:57 AM ET
636    Thursday, September 21, 2023 12:56 AM ET
637    Wednesday, September 20, 2023 1:11 AM ET
638     Tuesday, September 19, 2023 12:56 AM ET
639       Monday, September 18, 2023 1:05 AM ET
640       Friday, September 15, 2023 1:18 AM ET
641    Thursday, September 14, 2023 12:50 AM ET
642      Tuesday, September 12, 2023 1:23 AM ET
643       Monday, September 11, 2023 1:54 AM ET
644        Friday, September 8, 2023 1:27 AM ET
645     Thursday, September 7, 2023 12:34 AM ET
646     Wednesday, September 6, 2023 1:07 AM ET
647       Tuesday, September 5, 2023 1:05 AM ET
648        Friday, September 1, 2023 1:12 AM ET
649         Tuesday, August 29, 2023 1:08 AM ET
Name: publication_date, dtype: object
